# Python OOP Fundamentals

This notebook teaches Object-Oriented Programming (OOP) basics in Python by **building reusable classes** step-by-step.

By the end, we'll have a working mini card game using these classes:
- `Card`
- `Deck`
- `Hand`
- `Player`
- `Game`

Along the way we’ll learn:
- classes and objects
- `__init__` and attributes
- methods
- encapsulation (keeping logic inside the right class)
- composition (objects containing other objects)
- simple inheritance

## Why OOP?

OOP helps us organize code by bundling:
- **data** (attributes / state)
- **behavior** (methods)

Instead of scattered variables and functions, we model real concepts:
- A `Card` has a `rank` and `suit`
- A `Deck` *contains* many `Card`s and can `shuffle()` and `draw()`
- A `Player` has a `Hand` and can `play_card()`


In [2]:
# This is your class
class Austin:
    # This is a method
    def say_hello(self):
        print("Hello!")

# This is an object
a = Austin()
a.say_hello()

a2 = Austin()
a2.say_hello()


Hello!
Hello!


In [5]:
class Car:
    def drive(self):
        print("Vroom Vroom") 

tesla = Car()
mercedes = Car()

tesla.drive()

Vroom Vroom


## `__init__`, `self`, and attributes

- `__init__` runs when you create a new object.
- `self` refers to "this specific object."
- Attributes belong to the object (each object gets its own copy).


In [6]:
class Card:
    def __init__(self, rank, suit):
        self.rank = rank # attribute
        self.suit = suit # attribute

    def describe(self):
        return f"{self.rank} of {self.suit}"
    
c1 = Card("A", "Spades")
c2 = Card("10", "Hearts")

c1.describe()

'A of Spades'

The following is incorrect, since there is no 'self' in front of the attributes:

In [ ]:
class Dog:
    def __init__(breed, age):
        breed = breed
        age = age


## Encapsulation

Instead of writing logic *outside* the object…

Bad style:
```python
# scattered logic
if card.rank == "A": ...
```

Better OOP style: put logic inside the class and make the class responsible for its own behavior

Next: Teach `Card` how to compute its own value.

In [ ]:
class Card:
    RANK_VALUES = {
        "2": 2, "3": 3, "4": 4, "5": 5, "6": 6,
        "7": 7, "8": 8, "9": 9, "10": 10,
        "J": 11, "Q": 12, "K": 13, "A": 14
    }

    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def describe(self):
        return f"{self.rank} of {self.suit}"
    
    def value(self):
        return Card.RANK_VALUES[self.rank]